In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in the dataset:

In [2]:
train_df = pd.read_csv("../datasets/train.csv")

# Investigate the data:

### "Train" dataset:

In [3]:
train_df.shape

(2051, 81)

In [4]:
train_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

# Notes (Prior to cleaning):

### Differences between the sets:

- The test contains 1 less column than train: test does not contain "SalePrice"
    - The "SalesPrice" column contains all <b><i>"except for the target that we are trying to predict in our Regression model."</i></b>
- ~~The column names in both sets will need to be reformatted, to make them easier to work with.~~
    - <b>A function can be used to iterate through and automate necessary changes.</b>
- What are the missing values?
    - How many values should each col contain, including na?
        - <b>Train: 2051 rows
        - Test: 878 rows</b>
    - What cols have missing values?
        - <b>Train: there are 26 cols with NaN
        - Test: there are 19 cols with NaN</b>
    - What do these missing values mean?
        - <b>Missing values in some cols will mean an amenity or feature is not present at that particular parcel (i.e. does not have a pool, or garage, or fireplace, etc).
        - For the BSMT sq foot cols, NaN could be problematic. Does this unit not have a basement?
        - Could just lump these metrics all in together, as "has basement or not"</b>
- Variable types info (available via the "data description":
    - <b>Nominal: Named, categorical
        - nominal vars do not contribute to the LR (check in objects or not)
    - Ordinal: named, ranked
    - Discrete: dates, # full/half bathrooms, # bedrooms, # kitchens, # rooms 
    - Continuous: size (sq footage), $ value of misc. feature, SALE PRICE</b>
- Identify Outliers:
    - <b>Approach: (Could use a function for this)
        - Find 50, 25, and 75%'s for dataset
        - Anything < 1.5 * the IQR under Q1, or
        - Anything > 1.5 * the IQR above Q3</b>
- Age of house?

### Train Data:
- Lot area is continuous, == the size. Float would be better suited for this
- Dummies?:
    - MS Zoning
    - Street
    - Alley
    - Conditions 1 and 2
- NaN:
    - Alley type NaN can be converted to NA, as per the data doc

# Clean the data:

***Format the Column Names:***

In [6]:
def format_col(df):
    # To lowercase
    df.columns = map(str.lower, df.columns)  # <-- Referenced: https://chrisalbon.com/python/data_wrangling/pandas_lowercase_column_names/
    
    # Remove spaces, add "_":
    df.columns = df.columns.str.replace(" ", "_")
    
    
format_col(train_df)

train_df.rename(columns={"saleprice": "sale_price"}, inplace=True)

### NaN's

In [7]:
train_df.isna().sum()

id                0
pid               0
ms_subclass       0
ms_zoning         0
lot_frontage    330
               ... 
misc_val          0
mo_sold           0
yr_sold           0
sale_type         0
sale_price        0
Length: 81, dtype: int64

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2051 non-null   int64  
 1   pid              2051 non-null   int64  
 2   ms_subclass      2051 non-null   int64  
 3   ms_zoning        2051 non-null   object 
 4   lot_frontage     1721 non-null   float64
 5   lot_area         2051 non-null   int64  
 6   street           2051 non-null   object 
 7   alley            140 non-null    object 
 8   lot_shape        2051 non-null   object 
 9   land_contour     2051 non-null   object 
 10  utilities        2051 non-null   object 
 11  lot_config       2051 non-null   object 
 12  land_slope       2051 non-null   object 
 13  neighborhood     2051 non-null   object 
 14  condition_1      2051 non-null   object 
 15  condition_2      2051 non-null   object 
 16  bldg_type        2051 non-null   object 
 17  house_style   

In [9]:
# garage, bsmt have complicated setup to indicate they have or do not have garage/bsmt
# Could engineer a new col indicate if they merely have garage/bsmt

# Bsmt = finished vs. unfinished.

***fence***

In [10]:
# 400 non-null, NO NA listed in values_count
# add 0/ NA value?
train_df["fence"].fillna("NA", inplace=True)
train_df["fence"].value_counts()

NA       1651
MnPrv     227
GdPrv      83
GdWo       80
MnWw       10
Name: fence, dtype: int64

***bsmt cols***
- May be best to not include bsmt cols in the model, as there is a lot of discrepancy in the nan counts between cols
- Or, drop the 1 nan value in the col, and use that data (to indicate if has a basement, or create a has bsmt/ does not col)

In [11]:
# Total bsmt sq feet
# print(train_df.loc[train_df["total_bsmt_sf"].isna()])  # row 1327
train_df["total_bsmt_sf"].isna().sum()
train_df["total_bsmt_sf"].value_counts()
train_df["total_bsmt_sf"].dropna(inplace=True)

***misc cols***

In [12]:
# Total $0 misc values
train_df["misc_val"].value_counts()

# Number of nan in this col is equal to 0 above, can convert to type na
train_df["misc_feature"].isna().sum()

train_df["misc_feature"].unique()

train_df["misc_feature"].fillna("NA", inplace=True)
train_df["misc_feature"].isna().sum()

0

***pool cols***

- All nan in pool_qc corresponding to NA pool in pool_area, where pool_area was size 0.

In [13]:
print(train_df["pool_area"].value_counts())
print(train_df["pool_qc"].value_counts())

train_df["pool_qc"].fillna("NA", inplace=True)
train_df["pool_qc"].isna().sum()

0      2042
561       1
519       1
800       1
738       1
648       1
576       1
480       1
368       1
228       1
Name: pool_area, dtype: int64
Gd    4
Fa    2
TA    2
Ex    1
Name: pool_qc, dtype: int64


0

***mas_vr_type***

In [14]:
# The NaN could be equal to the Cinder Block type.
# Not much of anything to back this up though.
# Both are missing 22 values. Even if Type is changed to CBlock,
# no way of knowing the values for the area. Best to not to change value

train_df["mas_vnr_type"].dropna(inplace=True)
train_df["mas_vnr_area"].dropna(inplace=True)

train_df["mas_vnr_type"].isna().sum()

0

***lot_frontage***

In [37]:
# train_df.loc[train_df["lot_frontage"].isna()]  # <-- No linear feet of street connected to property?
# This could maybe just be == 0? The other lot metrics have no NaN values.

# May need to drop these, instead.
train_df["lot_frontage"].dropna(inplace=True)

In [38]:
train_df["lot_frontage"].isna().sum()

0

***alley***

In [34]:
# Fix alley col
# No NA values listed, although supposed to be in the value counts
train_df["alley"].fillna("NA", inplace=True)

train_df["alley"].unique()

array(['NA', 'Pave', 'Grvl'], dtype=object)

***garage cols***

In [18]:
train_df["garage_area"].dropna(inplace=True)

In [19]:
train_df["garage_area"].isna().sum()

0

***fireplace***

In [20]:
# If the number of units with 0 fireplaces == the number of nans in the fireplace_qu col, change to "NA"
# train_df["fireplace_qu"].unique()
# train_df["fireplaces"].unique()  
# train_df["fireplace_qu"].isna().sum()
# (train_df["fireplaces"] == 0).sum()

train_df["fireplace_qu"].fillna("NA", inplace=True)

train_df["fireplace_qu"].isna().sum()

0

***bsmt baths***
- Cannot tell if the nan rows should be zeroed or not...will be dropped.

In [21]:
train_df["bsmt_full_bath"].isna().sum()

train_df["bsmt_full_bath"].value_counts()

train_df["bsmt_half_bath"].isna().sum()

train_df["bsmt_half_bath"].value_counts()

train_df["bsmt_full_bath"].dropna(inplace=True)

train_df["bsmt_half_bath"].dropna(inplace=True)

### Convert Col type:

In [22]:
# Change lot area to type float
def col_to_float(df, col):
    return df[col].astype("float")

train_df["lot_area"] = col_to_float(train_df, "lot_area")

In [23]:
# Change bsmt_full_bath to int, its discrete
def col_to_int(df, col):
    return df[col].astype("int")

# train_df["bsmt_full_bath"] = col_to_int(train_df, "bsmt_full_bath")

In [24]:
def col_to_str(df, col):
    return df[col].astype("object")

# train_df["ms_subclass"] = col_to_str(train_df, "bsmt_full_bath")

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2051 non-null   int64  
 1   pid              2051 non-null   int64  
 2   ms_subclass      2051 non-null   int64  
 3   ms_zoning        2051 non-null   object 
 4   lot_frontage     1721 non-null   float64
 5   lot_area         2051 non-null   float64
 6   street           2051 non-null   object 
 7   alley            2051 non-null   object 
 8   lot_shape        2051 non-null   object 
 9   land_contour     2051 non-null   object 
 10  utilities        2051 non-null   object 
 11  lot_config       2051 non-null   object 
 12  land_slope       2051 non-null   object 
 13  neighborhood     2051 non-null   object 
 14  condition_1      2051 non-null   object 
 15  condition_2      2051 non-null   object 
 16  bldg_type        2051 non-null   object 
 17  house_style   

### Save the Clean CSV:

In [40]:
train_df.to_csv("../datasets/clean_train.csv")

In [41]:
train_df.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_price
0,109,533352170,60,RL,NaN,13517.0,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492.0,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922.0,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802.0,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235.0,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2010,WD,138500


In [39]:
train_df["lot_frontage"].isna().sum()

0

In [29]:
train_df.isna().sum()

id                0
pid               0
ms_subclass       0
ms_zoning         0
lot_frontage    330
               ... 
misc_val          0
mo_sold           0
yr_sold           0
sale_type         0
sale_price        0
Length: 81, dtype: int64

# Random / Old

In [30]:
# # bsmt_qual, bsmt_cond, bsmt_exposure, bsmtfin_type_1, bsmtfin_sf_1, bsmtfin_type_2
# # bsmtfin_sf_2, bsmt_unf_sf, total_bsmt_sf

# print(f"bmst qual value counts: \n{train_df['bsmt_qual'].value_counts()}")
# # NO NA
# # TA    887
# # Gd    864
# # Ex    184
# # Fa     60
# # Po      1
# print(f"bsmt qual nan: {train_df['bsmt_qual'].isna().sum()}")

# print(f"\nbsmt cond value counts: \n{train_df['bsmt_cond'].value_counts()}")
# # Same as above
# print(f"bsmt cond nan: {train_df['bsmt_cond'].isna().sum()}")

# print(f"\nbsmt exposure value counts: \n{train_df['bsmt_exposure'].value_counts()}")
# # No    1339
# # Av     288
# # Gd     203
# # Mn     163
# print(f"bsmt exposure nan: {train_df['bsmt_exposure'].isna().sum()}")

# train_df["bsmtfin_type_1"].value_counts()
# # Unf    603
# # ALQ    293
# # BLQ    200
# # Rec    183
# # LwQ    102
# train_df["bsmtfin_type_1"].isna().sum()  # 55 nan

# train_df["bsmtfin_sf_1"].value_counts()  # type 1 sq feet
# train_df["bsmtfin_sf_1"].isna().sum()  # 1 nan
# (train_df["bsmtfin_sf_1"] == 0.00).sum()

# train_df["bsmtfin_type_2"].value_counts()
# # Unf    1749
# # Rec      80
# # LwQ      60
# # BLQ      48
# # ALQ      35
# # GLQ      23
# train_df["bsmtfin_type_2"].isna().sum()  # 56 nan

# train_df["bsmtfin_sf_2"].value_counts()  # type 2 sq feet
# train_df["bsmtfin_sf_2"].isna().sum()  # 1 nan

# train_df["bsmt_unf_sf"].value_counts()  # unfinished sq feet of basement area
# train_df["bsmt_unf_sf"].isna().sum()  # 1 nan

# train_df["total_bsmt_sf"].value_counts()  # Total sq feet of basement
# train_df["total_bsmt_sf"].isna().sum()  # 1 nan

In [31]:
# # Row 1712 is NA on garage cols across the board. Should be dropped in all garage cols.

# train_df.loc[train_df["garage_cars"].isna()]

# train_df.loc[train_df["garage_area"].isna()]

# train_df["garage_cars"].dropna(inplace=True)
# train_df["garage_area"].dropna(inplace=True)

# # index_lst_1 = []
# # for index in train_df.index[(train_df["garage_cars"] == 0)]:
# #     print(index)
# #     index_lst_1.append(index)
# # # ^^ The indexes lined up between the nan and NA cols


# # index_lst_2 = []
# # for index in train_df.index[(train_df["garage_type"].isna())]:
# #     index_lst_2.append(index)


# # compare_lst = []
# # for i in index_lst_1:
# #     if i not in index_lst_2:
# #         compare_lst.append(i)
# # print(compare_lst_lst)

# train_df["garage_cars"].isna().sum()

# train_df["garage_type"].fillna("NA", inplace=True)